<a href="https://colab.research.google.com/github/paulaleite/aprendizagemDeMaquinaa2022/blob/main/Trabalho_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho Final para Disciplina de Aprendizagem de Máquina
### Paula Torales Leite
Textinho explicando o problema

## Importações

In [1]:
# Data processing
import pandas as pd
import numpy as np
import scipy.stats

# Similarity
from sklearn.metrics.pairwise import cosine_similarity

## Preparação dos Dados

In [28]:
# Read in data
ratings=pd.read_csv('./new_data/ratings.csv', sep=';')
# Take a look at the data
ratings.head()

,userID,challengeID,objectiveID,rating
0,24,4,30,2
1,8,6,25,3
2,7,3,31,2
3,30,4,5,4
4,27,7,49,1


In [29]:
# Get the dataset information
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   userID       833 non-null    int64
 1   challengeID  833 non-null    int64
 2   objectiveID  833 non-null    int64
 3   rating       833 non-null    int64
dtypes: int64(4)
memory usage: 26.2 KB


In [31]:
# Number of users
print('The ratings dataset has', ratings['userID'].nunique(), 'unique users')
# Number of movies
print('The ratings dataset has', ratings['challengeID'].nunique(), 'unique challenges')

print('The ratings dataset has', ratings['objectiveID'].nunique(), 'unique objectives')
# Number of ratings
print('The ratings dataset has', ratings['rating'].nunique(), 'unique ratings')
# List of unique ratings
print('The unique ratings are', sorted(ratings['rating'].unique()))

The ratings dataset has 30 unique users
The ratings dataset has 8 unique challenges
The ratings dataset has 55 unique objectives
The ratings dataset has 5 unique ratings
The unique ratings are [1, 2, 3, 4, 5]


In [34]:
# Read in data
objectives = pd.read_csv('./new_data/objectives.csv', sep=';')
# Take a look at the data
objectives.head()

,objectiveID,title
0,1,Objetivo1
1,2,Objetivo2
2,3,Objetivo3
3,4,Objetivo4
4,5,Objetivo5


In [35]:
# Merge ratings and movies datasets
df = pd.merge(ratings, objectives, on='objectiveID', how='inner')
# Take a look at the data
df.head()

,userID,challengeID,objectiveID,rating,title
0,24,4,30,2,Objetivo30
1,26,4,30,5,Objetivo30
2,12,2,30,2,Objetivo30
3,17,6,30,3,Objetivo30
4,12,2,30,1,Objetivo30


## Análise Exploratória

Colocar o que é duplicado e filtrar por quantidade avaliações. Só os objetivos que tem um número x de avaliações

In [43]:
# # Aggregate by movie
# agg_ratings = df.groupby('title').agg(mean_rating = ('rating', 'mean'),
#                                                 number_of_ratings = ('rating', 'count')).reset_index()
# # Keep the movies with over 3 ratings
# agg_ratings_GT100 = agg_ratings[agg_ratings['number_of_ratings']>1]
# # Check the information of the dataframe
# agg_ratings_GT100.info()

In [37]:
# Check popular movies
# agg_ratings_GT100.sort_values(by='number_of_ratings', ascending=False).head()

In [38]:
# Merge data
# df_GT100 = pd.merge(df, agg_ratings_GT100[['title']], on='title', how='inner')
# df_GT100.info()

In [39]:
# # Number of users
# print('The ratings dataset has', df_GT100['userId'].nunique(), 'unique users')
# # Number of movies
# print('The ratings dataset has', df_GT100['movieId'].nunique(), 'unique movies')
# # Number of ratings
# print('The ratings dataset has', df_GT100['rating'].nunique(), 'unique ratings')
# # List of unique ratings
# print('The unique ratings are', sorted(df_GT100['rating'].unique()))

## Criando a Matriz

In [81]:
current_challenge = 1
filtered_df = df[df.challengeID==current_challenge]
filtered_df.info()
print('The ratings dataset has', filtered_df['challengeID'].nunique(), 'unique challenges')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 17 to 827
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userID       96 non-null     int64 
 1   challengeID  96 non-null     int64 
 2   objectiveID  96 non-null     int64 
 3   rating       96 non-null     int64 
 4   title        96 non-null     object
dtypes: int64(4), object(1)
memory usage: 4.5+ KB
The ratings dataset has 1 unique challenges


In [82]:
# Create user-item matrix
matrix = filtered_df.pivot_table(index='title', columns='userID', values='rating')
matrix.head()

userID,1,2,3,4,5,6,7,8,9,10,...,20,21,22,23,24,25,26,28,29,30
title,,,,,,,,,,,,,,,,,,,,,
Objetivo1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
Objetivo11,NaN,NaN,NaN,5.0,NaN,NaN,NaN,1.0,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Normalização dos Dados

In [83]:
# Normalize user-item matrix
matrix_norm = matrix.subtract(matrix.mean(axis=1), axis = 0)
matrix_norm.head()

userID,1,2,3,4,5,6,7,8,9,10,...,20,21,22,23,24,25,26,28,29,30
title,,,,,,,,,,,,,,,,,,,,,
Objetivo1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
Objetivo11,NaN,NaN,NaN,2.333333,NaN,NaN,NaN,-1.666667,NaN,NaN,...,-0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Score de Similaridade

In [84]:
# Item similarity matrix using Pearson correlation
item_similarity = matrix_norm.T.corr()
item_similarity.head()

title,Objetivo1,Objetivo10,Objetivo11,Objetivo12,Objetivo13,Objetivo15,Objetivo16,Objetivo18,Objetivo19,Objetivo2,...,Objetivo49,Objetivo5,Objetivo51,Objetivo52,Objetivo53,Objetivo54,Objetivo6,Objetivo7,Objetivo8,Objetivo9
title,,,,,,,,,,,,,,,,,,,,,
Objetivo1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo10,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo11,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Previsão
Para um usuário

In [47]:
# # Pick a user ID
# picked_userid = 1
# # Pick a movie
# picked_movie = 'American Pie (1999)'
# # Movies that the target user has watched
# picked_userid_watched = pd.DataFrame(matrix_norm[picked_userid].dropna(axis=0, how='all')\
#                           .sort_values(ascending=False))\
#                           .reset_index()\
#                           .rename(columns={1:'rating'})
# picked_userid_watched.head()

In [48]:
# # Similarity score of the movie American Pie with all the other movies
# picked_movie_similarity_score = item_similarity[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})
# # Rank the similarities between the movies user 1 rated and American Pie.
# n = 5
# picked_userid_watched_similarity = pd.merge(left=picked_userid_watched, 
#                                             right=picked_movie_similarity_score, 
#                                             on='title', 
#                                             how='inner')\
#                                      .sort_values('similarity_score', ascending=False)[:n]
# # Take a look at the User 1 watched movies with highest similarity
# picked_userid_watched_similarity

In [49]:
# # Calculate the predicted rating using weighted average of similarity scores and the ratings from user 1
# predicted_rating = round(np.average(picked_userid_watched_similarity['rating'], 
#                                     weights=picked_userid_watched_similarity['similarity_score']), 6)
# print(f'The predicted rating for {picked_movie} by user {picked_userid} is {predicted_rating}' )

## Recomendação

In [85]:
# Item-based recommendation function
def item_based_rec(picked_userid=1, number_of_similar_items=5, number_of_recommendations =3):
  import operator
  # Movies that the target user has not watched
  picked_userid_unwatched = pd.DataFrame(matrix_norm[picked_userid].isna()).reset_index()
  picked_userid_unwatched = picked_userid_unwatched[picked_userid_unwatched[1]==True]['title'].values.tolist()
  # Movies that the target user has watched
  picked_userid_watched = pd.DataFrame(matrix_norm[picked_userid].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={1:'rating'})
  
  # Dictionary to save the unwatched movie and predicted rating pair
  rating_prediction ={}  
  # Loop through unwatched movies          
  for picked_movie in picked_userid_unwatched: 
    # Calculate the similarity score of the picked movie iwth other movies
    picked_movie_similarity_score = item_similarity[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})
    # Rank the similarities between the picked user watched movie and the picked unwatched movie.
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched, 
                                                right=picked_movie_similarity_score, 
                                                on='title', 
                                                how='inner')\
                                        .sort_values('similarity_score', ascending=False)[:number_of_similar_items]
    # Calculate the predicted rating using weighted average of similarity scores and the ratings from user 1
    predicted_rating = round(np.average(picked_userid_watched_similarity['rating'], 
                                        weights=picked_userid_watched_similarity['similarity_score']), 6)
    # Save the predicted rating in the dictionary
    if pd.isna(predicted_rating) == False:
      rating_prediction[picked_movie] = predicted_rating
    # Return the top recommended movies

  return sorted(rating_prediction.items(), key=operator.itemgetter(1), reverse=True)[:number_of_recommendations]

# Get recommendations
recommended_objectives = item_based_rec(picked_userid=1, number_of_similar_items=5, number_of_recommendations =3)
recommended_objectives

[]